In [1]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import numpy as np
from scipy import stats
matplotlib.rcParams['svg.fonttype'] = 'none'
matplotlib.rc('font', family='sans-serif')
matplotlib.rc('font', serif='Arial')
matplotlib.rc('text', usetex='false')
matplotlib.rc('pdf', fonttype=42)

In [2]:
## let's instead count all non-fixed variants in the data

##a wrapper class to store a parsed line from a VCF file
class Variant:
    chrome = ''
    pos = 0
    line = ''
    ##ref is in the 0 position, alts further
    nuc_seqs = []
    ##list of list of genotypes
    gts = []
    ##list of all GT quality scores
    quals = []
    ##reads supporting each allele
    read_depth = []

##takes a line from a VCF files and parses it
def parse_vcf_list(vcf_list):
    ##store the values in the Variant object
    cur_variant = Variant()
    cur_variant.chrome = vcf_list[0]
    cur_variant.pos = int(vcf_list[1])

    ##set the sequences for ref and alt, allowing for > 1 alt allele
    cur_ref= [vcf_list[3]]
    cur_alts = vcf_list[4].split(',')
    cur_seqs =cur_ref + cur_alts
    cur_variant.nuc_seqs = cur_seqs

    ##process the raw genotypes
    raw_gts = vcf_list[9:]
    cur_gts =  map(lambda x: x.split(':')[0].split('/'),raw_gts)
    #cur_gts =  map(lambda x: map(lambda y: int(y),x),cur_gts)
    cur_variant.gts = cur_gts
    cur_quals = []
    cur_rd_list = []
    for cur_raw_gt in raw_gts:
        cur_qual = 0
        cur_rd = []
        split = cur_raw_gt.split(':')
        if len(split) > 3:
            if split[3] == '.':
                cur_qual=0
                
            else:
                cur_qual = int(split[3])
                cur_rd = map(lambda x:int(x),split[1].split(','))
        cur_quals.append(cur_qual)
        cur_rd_list.append(cur_rd)
    #cur_quals = int(map(lambda x: x.split(':')[3],raw_gts))
    cur_variant.quals=cur_quals
    cur_variant.read_depth = cur_rd_list
    return cur_variant

##opens and parses a VCF file line by line
##yields a Variant object for each line
def VCF_parser(vcf_file):
    


    f = open(vcf_file,'r')
    
    for line in f:
        ##chomp dat header
        if line.startswith('#'):
            continue
            
        line = line.strip()
        split = line.split('\t')
        
        cur_variant = parse_vcf_list(split)
        cur_variant.line = line
        
        yield cur_variant
        

##counts non-fixed variants
def count_unfixed_variants(invcf,MIN_GT=0):     
    
    unique_vars = 0

    
    for cur_variant in VCF_parser(invcf):
        ##keep track of variants we have seen, if > 1, then the variant isn't fixed
        seen_vars = set()
        for i,genotype in enumerate(cur_variant.gts):
            for allele in genotype:
                cur_qual = cur_variant.quals[i]
                ##only continue if the variant is of sufficient quality
                if cur_qual < MIN_GT:
                    continue
                
                ##if the allele isn't a digit, it is missing, ignore it
                if not allele.isdigit():
                    continue
                
                
                seen_vars.add(allele)
        
        ##count all unfixed alleles (>1 allele can be unfixed)
        ##can't be less than 0
        cur_new_vars = max(len(seen_vars) - 1,0)
        if cur_new_vars >= 1:
            pass
            #print cur_variant.nuc_seqs
            #print cur_variant.gts
            #print cur_variant.chrome,cur_variant.pos
        unique_vars += cur_new_vars
    
    return unique_vars
        
DNA_sample_dict = {
    'AG07A'   : 'CERC',
    'AG07B'   : 'SAPC',
    'AG07C'   : 'GARC',
    'AG07D'   : 'BEPA;dad(BEPAxLITC)',
    'AG07E'   : 'LITC;mom(BEPAxLITC)',
    'AG07F'   : 'FRI;dad(FRIxLITC)',
    'AG08A'   : 'FTC;dad(FTCxLITC)',
    'AG08B'   : 'LITC;mom(mom FTCxLITC)',
    'AG08C'   : 'LITC;dad(LITCxLITC 3)',
    'AG08D'   : 'LITC;mom(PAXBxLITC 28)',
    'AG08E'   : 'LITC;mom(PAXBxLITC 29)',
    'AG08F'   : 'PAXB;dad(PAXBxLITC 28/29)',
    'AMG'     : 'FTC;mom(BigRev) ',
    'AMG01'   : 'LITC;dad(BigRev)',
    'JCH002A' : 'PAXB;dad(PAXBXRABS)',
    'JCH002B' : 'PAXB;dad(PAXBXJAMA-ANTHONY)',
    'JCH002C' : 'PAXB;dad(PAXBxJAMA-LYNDA)',
    'JCH002D' : 'PAXB;dad(WV)',
    'JCH002E' : 'PAXB;dad(JCH_ASE)',
    'JCH002F' : 'RABS;mom(JCH_DNaseI)',
    'JCH006A' : 'CERC;JCH053113-1M)',
    'JCH006B' : 'CERC;AML110812-1M',
    'JCH006C' : 'JAMA;PSL040511-2F',
    'JCH006D' : 'Hutu F3',
    'JCH006E' : 'CCD 2007',
    'JCH007A' : 'LITC LCM || benthic',
    'JCH007B' : 'EnosB || benthic',
    'JCH007C' : 'PriestB || benthic',
    'JCH007D' : 'PaxtonB || benthic',
    'PCAG5'   : 'PAXBxLITC;F2 (21=D)',
    'PCAG6'   : 'PAXBxLITC;F2 (21=B)',
    'PCAG7'   : 'CF_FTC;mom',
    'PCAG8'   : 'CF_LITC;dad',
    'JCH009G' : 'RABS;mom (CERC x RABS F2 Mom)'
}

PHYLIP_sample_dict = {
    'AG07A'   : 'CERC-wd454',
    'AG07B'   : 'SAPC-wd448',
    'AG07C'   : 'GARC-wd711',
    'AG07D'   : 'BEPA-wdBxL',
    'AG07E'   : 'LITC-wmBxL',
    'AG07F'   : 'FRI-wdFRxL',
    'AG08A'   : 'FTC-wdFTxL',
    'AG08B'   : 'LITC-mFTxL',
    'AG08C'   : 'LITC-wdLC3',
    'AG08D'   : 'LITC-wmL28',
    'AG08E'   : 'LITC-wmL29',
    'AG08F'   : 'PAXB-wdPxL',
    'AMG'     : 'FTC-lmBRev',
    'AMG01'   : 'LITC-ldBRv',
    'JCH002A' : 'PAXB-ldPxR',
    'JCH002B' : 'PAXB-dPxJa',
    'JCH002C' : 'PAXB-dPxJl',
    'JCH002D' : 'PAXB-dWVir',
    'JCH002E' : 'PAXB-dJASE',
    'JCH002F' : 'RABS-mDNas',
    'JCH006A' : 'CERC-dCASE',
    'JCH006B' : 'CERC-dCxRB',
    'JCH006C' : 'JAMA-mPxJl',
    'JCH006D' : 'Hutu F3   ',
    'JCH006E' : 'CCD 2007  ',
    'JCH007A' : 'LITC-wmPB ',
    'JCH007B' : 'EnosB-wdPB',
    'JCH007C' : 'PriestB-wd',
    'JCH007D' : 'PaxtonB-wd',
    'PCAG5'   : 'PxLF2;21=D',
    'PCAG6'   : 'PxLF2;21=B',
    'PCAG7'   : 'CF_FTC;mom',
    'PCAG8'   : 'CF_LITCdad',
    'JCH009G' : 'RABS-lmCxR'
}

In [3]:
##filters invcf, gets rid of variants where less that PROP_PASS of samples have MIN_RD and MIN_GQ requirements
##Will keep header lines if HEADER=True
def filter_vcf(invcf,outvcf,HEADER=True,MIN_RD=3,MIN_GQ=5,PROP_FAIL=0,ONLY_SNPs=True,DIALLELE=True):
    
    f = open(invcf,'r')
    w = open(outvcf,'w')
    genome_ids = []
    if HEADER:
        for line in f:
            if line.startswith('##'):
                w.write(line)
            elif line.startswith('#'):
                w.write(line)
                genome_ids = line.strip().split()[9:]
            else:
                break
        f.close()      
        f = open(invcf,'r')
    print genome_ids  
    low_qual = 0
    low_rd = 0
    both = 0
    hq = 0
    total = 0
    indel = 0
    total_rd = []
    total_qual = []
    for cur_variant in VCF_parser(invcf):
        ##remove chrUn and chrXIX (the X chromosome)
        if cur_variant.chrome == 'chrUn' or cur_variant.chrome == 'chrXIX':
            continue
        ##We want total read depth, not any specific variant
        cur_variant.read_depth = map(sum,cur_variant.read_depth) 
        
        
         
        
        read_depth_array = np.array(cur_variant.read_depth)
        if total_rd == []:
            total_rd = read_depth_array
        else:
            total_rd = total_rd + read_depth_array
        qual_array = np.array(cur_variant.quals)
        if total_qual == []:
            total_qual = qual_array
        else:
            total_qual = total_qual + qual_array
        
        low_rd_sample_num =  len(read_depth_array[read_depth_array < MIN_RD])
        low_qual_sample_num =  len(qual_array[qual_array < MIN_GQ]) 
        
        ##minimum number of low quality samples per variant
        min_low_num = len(read_depth_array) * PROP_FAIL
        
        
        total =  hq + both + low_rd + low_qual 
        
        
        
        IS_SNP=True
        
        for genotype in cur_variant.nuc_seqs:
            if len(genotype) > 1:
                IS_SNP = False
                
        if not IS_SNP:
            indel +=1
            if ONLY_SNPs:
                continue
                
        IS_MULTI = False
        for genotype in cur_variant.gts:
            for allele in genotype:
                if allele != '.' and allele != '0' and allele != '1':
                    IS_MULTI = True
                    
        if DIALLELE and IS_MULTI:
            continue
        
        '''if total > 0 and (total % 100000) == 0:
            print cur_variant.chrome,low_rd,low_qual,both,hq,indel
            print total_rd/total
            print total_qual/total
        '''
        if low_rd_sample_num > min_low_num and low_qual_sample_num > min_low_num:
            both += 1
            continue
        elif low_rd_sample_num >min_low_num:
            low_rd += 1
            continue
        elif low_qual_sample_num > min_low_num:
            low_qual += 1
            continue
        else:
            
            hq +=1
            w.write(cur_variant.line + '\n')
            
        total =  hq + both + low_rd + low_qual
        
        
                    
        
            
            
        
    print low_rd,low_qual,both,hq,indel
    print total_rd/total
    print total_qual/total
        
        
        
    f.close()
    w.flush()
    w.close()
#filter_vcf('MOAV_071817_tested_haplotype.vcf','MOAV_071817_tested_filtered_haplotype.vcf')    
'''
['AG07A', 'AG07D', 'AG07E', 'AG07F', 'AG08A', 'AG08B', 'AG08D', 'AG08E', 'AG08F', 'AMG', 'AMG01', 'JCH002A', 'JCH002B', 'JCH002C', 'JCH002D', 'JCH006B', 'JCH006C', 'JCH006D', 'JCH006E', 'JCH007A', 'JCH007B', 'JCH007C', 'JCH007D']
[ 8  7  7  6  8  9 14 13  9 60 59  4  6  8  7  5  8  3  5 11 11 12  9]
[ 31  25  26  20  30  32  46  45  32 109 108  13  21  25  22  18  26  11
  18  37  36  38  30]
1092385 397985 6308942 857751 2044600'''

"\n['AG07A', 'AG07D', 'AG07E', 'AG07F', 'AG08A', 'AG08B', 'AG08D', 'AG08E', 'AG08F', 'AMG', 'AMG01', 'JCH002A', 'JCH002B', 'JCH002C', 'JCH002D', 'JCH006B', 'JCH006C', 'JCH006D', 'JCH006E', 'JCH007A', 'JCH007B', 'JCH007C', 'JCH007D']\n[ 8  7  7  6  8  9 14 13  9 60 59  4  6  8  7  5  8  3  5 11 11 12  9]\n[ 31  25  26  20  30  32  46  45  32 109 108  13  21  25  22  18  26  11\n  18  37  36  38  30]\n1092385 397985 6308942 857751 2044600"

In [21]:
##converts two nucleotides into a single letter (e.g. A, C -> M)
##This seems like it will work for DNAML
def convertDiNT(nuc1,nuc2):
    
    NTs = [nuc1,nuc2]
    
    if 'A' in NTs and 'C' in NTs:
        return 'M'
    if 'A' in NTs and 'G' in NTs:
        return 'R'
    if 'A' in NTs and 'T' in NTs:
        return 'W'
    if 'C' in NTs and 'G' in NTs:
        return 'S'
    if 'C' in NTs and 'T' in NTs:
        return 'Y'
    if 'G' in NTs and 'T' in NTs:
        return 'K'
    else:
        return nuc1


##makes a VCF file into a tsv matrix, usable for PCA
def vcf_to_gts(invcf,outgt,CHROME='',START=0,STOP=0):
    
    w = open(outgt,'w')
    f = open(invcf,'r')
    
    for line in f:
        line = line.strip()
        if line.startswith('##'):
            continue
        split = line.split()
        ids = split[9:]
        names = map(PHYLIP_sample_dict.get,ids,ids)
        break
    print ids,names
    f.close()
    header_list = ['chr:pos'] + names
    w.write('\t'.join(header_list) + '\n')
    
    for cur_variant in VCF_parser(invcf):
        
        if CHROME and START and STOP:
            
            if cur_variant.chrome != CHROME:
                continue
            if cur_variant.pos < START or cur_variant.pos > STOP:
                continue

        ##collapse scores into single number
        cur_gt_scores = map(lambda x: str(int(x[0]) + int(x[1])),cur_variant.gts)
        
        cur_chrpos = cur_variant.chrome + ':' + str(cur_variant.pos)
        
        new_list_list = [cur_chrpos] + cur_gt_scores
        w.write('\t'.join(new_list_list) + '\n')
    w.flush()
    w.close()
        
        
        
    

##makes a VCF file into a seq file, for use with the PHYLIP package.
def vcf_to_seqs(invcf,outseq,CHROME='',START=0,STOP=0):
    
    w = open(outseq,'w')
    f = open(invcf,'r')
    
    for line in f:
        line = line.strip()
        if line.startswith('##'):
            continue
        split = line.split()
        ids = split[9:]
        names = map(PHYLIP_sample_dict.get,ids,ids)
        break
    print ids,names
    f.close()
    seqlist = [[] for i in range(len(names))]
    var_count = 0
    for cur_variant in VCF_parser(invcf):
        if CHROME and START and STOP:
            
            if cur_variant.chrome != CHROME:
                continue
            if cur_variant.pos < START or cur_variant.pos > STOP:
                continue
        INDEL = False
        for var_seq in cur_variant.nuc_seqs:
            if len(var_seq) > 1:
               INDEL = True 
        if INDEL:
            continue
        ##The dreaded double lambda function. This turns the nested string gts into ints
        #genotypes = map(lambda x: map(lambda y: int(y),x),cur_variant.gts)
        var_count += 1
        for i,geno in enumerate(cur_variant.gts):
            if '.' in geno:
                seqlist[i].append('N')
                continue
            geno = map(lambda x: int(x),geno)
            seq1 = cur_variant.nuc_seqs[geno[0]]
            seq2 = cur_variant.nuc_seqs[geno[1]]
            ##Test if the sequences are equal - if not, give the dinucleotide code
            if seq1 == seq2:
                if seq1 == '*':
                    seq1 = 'N'
                seqlist[i].append(seq1)
            else:
                dinuc = convertDiNT(seq1,seq2)
                seqlist[i].append(dinuc)
            
            
    
    for i in range(len(seqlist)):
        seqlist[i] = ''.join(seqlist[i])
    
    
    ##write the seq outfile
    header = '  '+str(len(names))+'\t'+str(var_count)
    w.write(header + '\n')
    for i in range(len(names)):
        w.write(names[i] + '\t' + seqlist[i] + '\n')
        
    w.flush()
    w.close()

    
    
    
#vcf_to_gts('MOAV_071817_tested_filtered_haplotype.vcf','MOAV_071817_tested_filtered_haplotype.gt')   
#vcf_to_seqs('MOAV_071817_tested_filtered_haplotype.vcf','MOAV_071817_tested_filtered_haplotype.seq')   

#vcf_to_gts('MOAV_071817_tested_filtered_haplotype.vcf','MOAV_071817_tested_filtered_haplotype_I4.gt',CHROME='chrXXI',START=3853685,STOP=3858045)
#vcf_to_seqs('MOAV_071817_tested_filtered_haplotype.vcf','MOAV_071817_tested_filtered_haplotype_I4.seq',CHROME='chrXXI',START=3853685,STOP=3858045)
#vcf_to_seqs('MOAV_071817_tested_haplotype.vcf','MOAV_071817_tested_haplotype_I4.seq',CHROME='chrXXI',START=3853685,STOP=3858045)
#vcf_to_seqs('MOAV_071817_tested_haplotype.vcf','MOAV_071817_tested_haplotype_I4_min.seq',CHROME='chrXXI',START=3856000,STOP=3856500)




['AG07A', 'AG07D', 'AG07E', 'AG07F', 'AG08A', 'AG08B', 'AG08D', 'AG08E', 'AG08F', 'AMG', 'AMG01', 'JCH002A', 'JCH002B', 'JCH002C', 'JCH002D', 'JCH006B', 'JCH006C', 'JCH006D', 'JCH006E', 'JCH007A', 'JCH007B', 'JCH007C', 'JCH007D', 'JCH009G'] ['CERC-wd454', 'BEPA-wdBxL', 'LITC-wmBxL', 'FRI-wdFRxL', 'FTC-wdFTxL', 'LITC-mFTxL', 'LITC-wmL28', 'LITC-wmL29', 'PAXB-wdPxL', 'FTC-lmBRev', 'LITC-ldBRv', 'PAXB-ldPxR', 'PAXB-dPxJa', 'PAXB-dPxJl', 'PAXB-dWVir', 'CERC-dCxRB', 'JAMA-mPxJl', 'Hutu F3   ', 'CCD 2007  ', 'LITC-wmPB ', 'EnosB-wdPB', 'PriestB-wd', 'PaxtonB-wd', 'RABS-lmCxR']
['AG07A', 'AG07D', 'AG07E', 'AG07F', 'AG08A', 'AG08B', 'AG08D', 'AG08E', 'AG08F', 'AMG', 'AMG01', 'JCH002A', 'JCH002B', 'JCH002C', 'JCH002D', 'JCH006B', 'JCH006C', 'JCH006D', 'JCH006E', 'JCH007A', 'JCH007B', 'JCH007C', 'JCH007D', 'JCH009G'] ['CERC-wd454', 'BEPA-wdBxL', 'LITC-wmBxL', 'FRI-wdFRxL', 'FTC-wdFTxL', 'LITC-mFTxL', 'LITC-wmL28', 'LITC-wmL29', 'PAXB-wdPxL', 'FTC-lmBRev', 'LITC-ldBRv', 'PAXB-ldPxR', 'PAXB-dPxJa'